# Day 18: Many-Worlds Interpretation

## Part 1

As you approach Neptune, a planetary security system detects you and activates 
a giant tractor beam on Triton! You have no choice but to land.

A scan of the local area reveals only one interesting feature: a massive 
underground vault. You generate a map of the tunnels (your puzzle input). The 
tunnels are too narrow to move diagonally.

Only one **entrance** (marked `@`) is present among the **open passages** 
(marked `.`) and stone walls (#), but you also detect an assortment of 
**keys** (shown as lowercase letters) and **doors** (shown as uppercase 
letters). Keys of a given letter open the door of the same letter: `a` opens 
`A`, `b` opens `B`, and so on. You aren't sure which key you need to disable 
the tractor beam, so you'll need to **collect all of them**.

For example, suppose you have the following map:

```
#########
#b.A.@.a#
#########
```

Starting from the entrance (`@`), you can only access a large door (`A`) and 
a key (`a`). Moving toward the door doesn't help you, but you can move `2` steps 
to collect the key, unlocking `A` in the process:

```
#########
#b.....@#
#########
```

Then, you can move `6` steps to collect the only other key, `b`:

```
#########
#@......#
#########
```

So, collecting every key took a total of **`8`** steps.

Here is a larger example:

```
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################
```

The only reasonable move is to take key `a` and unlock door `A`:

```
########################
#f.D.E.e.C.b.....@.B.c.#
######################.#
#d.....................#
########################
```

Then, do the same with key `b`:

```
########################
#f.D.E.e.C.@.........c.#
######################.#
#d.....................#
########################
```

...and the same with key `c`:

```
########################
#f.D.E.e.............@.#
######################.#
#d.....................#
########################
```

Now, you have a choice between keys `d` and `e`. While key `e` is closer, 
collecting it now would be slower in the long run than collecting key `d` 
first, so that's the best choice:

```
########################
#f...E.e...............#
######################.#
#@.....................#
########################
```

Finally, collect key `e` to unlock door `E`, then collect key `f`, taking a 
grand total of **`86`** steps.

Here are a few more examples:

- 
```
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################
```

Shortest path is `132` steps: `b, a, c, d, f, e, g`

- 
```
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
```

Shortest paths are `136` steps;
one is: `a, f, b, j, g, n, h, d, l, o, e, p, c, i, k, m`

- 
```
########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################
```

Shortest paths are `81` steps; one is: `a, c, f, i, d, g, b, e, h`

**How many steps is the shortest path that collects all of the keys?**

In [1]:
from pathlib import Path

import numpy as np

input_path = Path("..") / "inputs" / "day_18.txt"
inputs = input_path.read_text()
inputs = inputs.split("\n")
inputs = [list(line) for line in inputs]
input_arr = np.array(inputs)

In [2]:
from collections import OrderedDict

In [3]:
import string

def find_shortest_path(maze):
    arr = np.array(maze)
    num_keys = np.isin(arr, list(string.ascii_lowercase)).sum()
    current_location = np.where(arr == "@")
    current_location = (current_location[0][0], current_location[1][0])
    queue = [(current_location, tuple(), 0),]
    
    visited = set()

    while True:
        current_location, keys, steps = queue.pop(0)
        
        current_val = arr[current_location]
        if current_val in string.ascii_lowercase:
            keys = tuple(sorted({*keys, current_val}))    
        elif current_val in string.ascii_uppercase and current_val.lower() not in keys:
            continue
        elif (current_location, keys) in visited:
            continue
        visited.add((current_location, keys))
        p = (
            (current_location[0], current_location[1]+1),
            (current_location[0], current_location[1]-1),
            (current_location[0]+1, current_location[1]),
            (current_location[0]-1, current_location[1])
        )
        p = (i for i in p if (i, keys) not in visited)
        p = (i for i in p if arr[i] != "#")
        p = (i for i in p if arr[i] in [*string.ascii_lowercase, ".", *[i.upper() for i in keys], "@", *keys])
        p = ((i, keys, steps+1) for i in p)
        queue.extend(p)

        if len(keys) == num_keys:
            return steps

### Test Cases

In [11]:
maze = ["#########","#b.A.@.a#","#########"]
maze_dict = {}
for y, line in enumerate(maze):
    for x, val in enumerate(line):
        maze_dict[(x, y)] = val
start = [k for k, v in maze_dict.items() if v == "@"][0]
num_keys = sum(i in string.ascii_lowercase for i in maze_dict.values())

queue = [[start, tuple(), 0]]
visited = {}

while len(queue):
    location, keys, length = queue.pop(0)
#     print(f"at {location}")
    if (location, keys) in visited:
#         print("but I've been here")
        continue
    else:
        visited[(location, keys)] = length + 1
    for direction in [0, 1]:
        for step in [-1, 1]:
            those_coords = list(location)
            those_coords[direction] += step
            those_coords = tuple(those_coords)
#             print(f"looking at {those_coords}")
            val_there = maze_dict[those_coords]
            
            if val_there == "#":
#                 print("dead end")
                continue
            elif val_there == "." or val_there == "@":
#                 print("can go here")
                queue.append([those_coords, keys, length + 1])
            elif val_there in string.ascii_uppercase:
#                 print(f"found gate {val_there}")
                if val_there.lower() in keys:
#                     print(f"but I have a key")
                    queue.append([those_coords, keys, length + 1])
            elif val_there in string.ascii_lowercase:
#                 print(f"found key {val_there}")
                if val_there in keys:
#                     print("but we already have it")
                    continue
                else:
                    new_keys = tuple(sorted([*keys, val_there]))
                    queue.append([those_coords, new_keys, length+1])

                    if len(new_keys) == num_keys:
                        print(length + 1)
                        raise ValueError
#     print("break")
                


8


ValueError: 

In [118]:
def do_it(maze):
    maze_dict = {}
    for y, line in enumerate(maze):
        for x, val in enumerate(line):
            maze_dict[(x, y)] = val
    start = [k for k, v in maze_dict.items() if v == "@"][0]
    num_keys = sum(i in string.ascii_lowercase for i in maze_dict.values())

    queue = [[start, tuple(), 0]]
    visited = {}

    while len(queue):
        location, keys, length = queue.pop(0)
#         print(f"at {location} with keys {keys}")
        if (location, keys) in visited:
#             print("but I've been here")
            continue
        else:
            visited[(location, keys)] = length + 1
        
        normals = [
                    ".", 
                    "@", 
                    *map(lambda x: x.upper(), keys),
                    *keys
                ]
        coordss = [
            (location[0]+1, location[1]), 
            (location[0]-1, location[1]),
            (location[0], location[1]-1),
            (location[0], location[1]+1)
        ]
        for those_coords in coordss:
                val_there = maze_dict[those_coords]


                
                if val_there in normals:
#                     print("can go here")
                    new_keys = keys
                    queue.append([those_coords, new_keys, length + 1])
                elif val_there in string.ascii_lowercase:
                    new_keys = tuple(sorted([*keys, val_there]))
                    queue.append([those_coords, new_keys, length+1])

                    if len(new_keys) == num_keys:
                        return(length + 1)
                        raise ValueError
#                 queue.append([those_coords, new_keys, length+1])


In [119]:
test_cases = [
    (["#########","#b.A.@.a#","#########"], 8),
    ([
        "012345678901234567890123"
        "########################",
        "#f.D.E.e.C.b.A.@.a.B.c.#",
        "######################.#",
        "#d.....................#",
        "########################"
    ], 86),
    ([
        "########################",
        "#...............b.C.D.f#",
        "#.######################",
        "#.....@.a.B.c.d.A.e.F.g#",
        "########################"
    ], 132),
    ([
        "#################",
        "#i.G..c...e..H.p#",
        "########.########",
        "#j.A..b...f..D.o#",
        "########@########",
        "#k.E..a...g..B.n#",
        "########.########",
        "#l.F..d...h..C.m#",
        "#################"
    ], 136),
    ([
        "########################",
        "#@..............ac.GI.b#",
        "###d#e#f################",
        "###A#B#C################",
        "###g#h#i################",
        "########################"
    ], 81)
    
]

for test_input, expected_output in test_cases:
#     test_input = [list(line) for line in test_input]
#     assert find_shortest_path(test_input) == expected_output
    r = do_it(test_input)
    assert r == expected_output

In [120]:
import datetime

print(datetime.datetime.now())

print(do_it(inputs))
print(datetime.datetime.now())


2020-04-15 14:51:39.443993
4954
2020-04-15 14:53:23.532655


In [124]:
%load_ext line_profiler
%lprun -f do_it do_it(test_input)


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-06 s

Total time: 0.297466 s
File: <ipython-input-118-53536c0beff0>
Function: do_it at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def do_it(maze):
     2         1          9.0      9.0      0.0      maze_dict = {}
     3         7         67.0      9.6      0.0      for y, line in enumerate(maze):
     4       150       3126.0     20.8      1.1          for x, val in enumerate(line):
     5       144       1812.0     12.6      0.6              maze_dict[(x, y)] = val
     6         1         93.0     93.0      0.0      start = [k for k, v in maze_dict.items() if v == "@"][0]
     7         1        220.0    220.0      0.1      num_keys = sum(i in string.ascii_lowercase for i in maze_dict.values())
     8                                           
     9         1         17.0     17.0      0.0      queue = [[start, tuple(), 0]]
    10         1         20.0     20.0      0.0      visited = {}
 

Answer to Part 1: **4954**

## Part 2

You arrive at the vault only to discover that there is not one vault, but **four** - each with its own entrance.

On your map, find the area in the middle that looks like this:

```
...
.@.
...
```

Update your map to instead use the correct data:

```
@#@
###
@#@
```

This change will split your map into four separate sections, each with its own entrance:

```
#######       #######
#a.#Cd#       #a.#Cd#
##...##       ##@#@##
##.@.##  -->  #######
##...##       ##@#@##
#cB#Ab#       #cB#Ab#
#######       #######
```

Because some of the keys are for doors in other vaults, it would take much too long to collect all of the keys by yourself. Instead, you deploy four remote-controlled robots. Each starts at one of the entrances (`@`).

Your goal is still to `collect all of the keys in the fewest steps`, but now, each robot has its own position and can move independently. You can only remotely control a single robot at a time. Collecting a key instantly unlocks any corresponding doors, regardless of the vault in which the key or door is found.

For example, in the map above, the top-left robot first collects key a, unlocking door `A` in the bottom-right vault:

```
#######
#@.#Cd#
##.#@##
#######
##@#@##
#cB#.b#
#######
```

Then, the bottom-right robot collects key `b`, unlocking door `B` in the bottom-left vault:

```
#######
#@.#Cd#
##.#@##
#######
##@#.##
#c.#.@#
#######
```

Then, the bottom-left robot collects key `c`:

```
#######
#@.#.d#
##.#@##
#######
##.#.##
#@.#.@#
#######
```

Finally, the top-right robot collects key `d`:

```
#######
#@.#.@#
##.#.##
#######
##.#.##
#@.#.@#
#######
```

In this example, it only took `8` steps to collect all of the keys.

Sometimes, multiple robots might have keys available, or a robot might have to wait for multiple keys to be collected:

```
###############
#d.ABC.#.....a#
######@#@######
###############
######@#@######
#b.....#.....c#
###############
```

First, the top-right, bottom-left, and bottom-right robots take turns collecting keys `a`, `b`, and `c`, a total of `6 + 6 + 6 = 18` steps. Then, the top-left robot can access key `d`, spending another `6` steps; collecting all of the keys here takes a minimum of **`24`** steps.

Here's a more complex example:

```
#############
#DcBa.#.GhKl#
#.###@#@#I###
#e#d#####j#k#
###C#@#@###J#
#fEbA.#.FgHi#
#############
```

- Top-left robot collects key `a`.
- Bottom-left robot collects key `b`.
- Top-left robot collects key `c`.
- Bottom-left robot collects key `d`.
- Top-left robot collects key `e`.
- Bottom-left robot collects key `f`.
- Bottom-right robot collects key `g`.
- Top-right robot collects key `h`.
- Bottom-right robot collects key `i`.
- Top-right robot collects key `j`.
- Bottom-right robot collects key `k`.
- Top-right robot collects key `l`.

In the above example, the fewest steps to collect all of the keys is **`32`**.

Here's an example with more choices:

```
#############
#g#f.D#..h#l#
#F###e#E###.#
#dCba@#@BcIJ#
#############
#nK.L@#@G...#
#M###N#H###.#
#o#m..#i#jk.#
#############
```

One solution with the fewest steps is:

- Top-left robot collects key `e`.
- Top-right robot collects key `h`.
- Bottom-right robot collects key `i`.
- Top-left robot collects key `a`.
- Top-left robot collects key `b`.
- Top-right robot collects key `c`.
- Top-left robot collects key `d`.
- Top-left robot collects key `f`.
- Top-left robot collects key `g`.
- Bottom-right robot collects key `k`.
- Bottom-right robot collects key `j`.
- Top-right robot collects key `l`.
- Bottom-left robot collects key `n`.
- Bottom-left robot collects key `m`.
- Bottom-left robot collects key `o`.

This example requires at least **`72`** steps to collect all keys.

After updating your map and using the remote-controlled robots, **what is the fewest steps necessary to collect all of the keys?**

## Test Cases